<a href="https://colab.research.google.com/github/bhartiansh/cnn_pruning_cifar10/blob/main/SNIP_ResNet20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/bhartiansh/cnn_pruning_cifar10.git
%cd cnn_pruning_cifar10

!pip install -q tensorflow-model-optimization

Cloning into 'cnn_pruning_cifar10'...
remote: Enumerating objects: 206, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 206 (delta 32), reused 2 (delta 2), pack-reused 144 (from 1)
Receiving objects: 100% (206/206), 124.13 KiB | 676.00 KiB/s, done.
Resolving deltas: 100% (95/95), done.
/content/cnn_pruning_cifar10


In [2]:
import tensorflow as tf
import numpy as np
from models.resnet20 import build_resnet20

def compute_snip_scores(model, x_sample, y_sample):
    with tf.GradientTape() as tape:
        logits = model(x_sample, training=True)
        loss = tf.keras.losses.sparse_categorical_crossentropy(y_sample, logits)
        loss = tf.reduce_mean(loss)

    grads = tape.gradient(loss, model.trainable_weights)
    scores = []

    for w, g in zip(model.trainable_weights, grads):
        if 'kernel' in w.name:
            scores.append(tf.reshape(tf.abs(w * g), [-1]))

    return tf.concat(scores, axis=0)

def apply_snip_mask(model, x_sample, y_sample, sparsity=0.5):
    scores = compute_snip_scores(model, x_sample, y_sample)
    k = int((1 - sparsity) * scores.shape[0])
    threshold = tf.sort(scores)[k]

    for w in model.trainable_weights:
        if 'kernel' in w.name:
            mask = tf.cast(tf.abs(w) >= threshold, tf.float32)
            w.assign(w * mask)

def quick_train(model, x_train, y_train, x_test, y_test, epochs=10, batch_size=64):
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              batch_size=batch_size,
              epochs=epochs,
              verbose=2)
    return model

In [3]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.3)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

170498071/170498071 [==============================] - 4s 0us/step
Epoch 1/30
782/782 - 61s - loss: 1.3401 - accuracy: 0.5109 - val_loss: 1.1363 - val_accuracy: 0.5894 - 61s/epoch - 78ms/step
Epoch 2/30
782/782 - 54s - loss: 0.9124 - accuracy: 0.6764 - val_loss: 1.2805 - val_accuracy: 0.5745 - 54s/epoch - 69ms/step
Epoch 3/30
782/782 - 57s - loss: 0.7387 - accuracy: 0.7416 - val_loss: 1.0174 - val_accuracy: 0.6621 - 57s/epoch - 73ms/step
Epoch 4/30
782/782 - 57s - loss: 0.6244 - accuracy: 0.7817 - val_loss: 0.7847 - val_accuracy: 0.7267 - 57s/epoch - 74ms/step
Epoch 5/30
782/782 - 55s - loss: 0.5460 - accuracy: 0.8086 - val_loss: 1.1357 - val_accuracy: 0.6390 - 55s/epoch - 70ms/step
Epoch 6/30
782/782 - 54s - loss: 0.4848 - accuracy: 0.8329 - val_loss: 0.7083 - val_accuracy: 0.7633 - 54s/epoch - 69ms/step
Epoch 7/30
782/782 - 54s - loss: 0.4294 - accuracy: 0.8519 - val_loss: 0.7287 - val_accuracy: 0.7584 - 54s/epoch - 69ms/step
Epoch 8/30
782/782 - 54s - loss: 0.3783 - accuracy: 0.8685

In [4]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.5)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

Epoch 1/30
782/782 - 61s - loss: 1.3626 - accuracy: 0.5041 - val_loss: 1.2454 - val_accuracy: 0.5578 - 61s/epoch - 79ms/step
Epoch 2/30
782/782 - 54s - loss: 0.9403 - accuracy: 0.6639 - val_loss: 1.0378 - val_accuracy: 0.6401 - 54s/epoch - 69ms/step
Epoch 3/30


KeyboardInterrupt: 

In [ ]:
# === Run it ===
if __name__ == "__main__":
    # Load CIFAR-10
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0
    y_train, y_test = y_train.flatten(), y_test.flatten()

    # Build model
    model = build_resnet20()

    # Run dummy forward pass to initialize weights
    model(tf.convert_to_tensor(x_train[:1]), training=False)

    # SNIP pruning using small batch
    apply_snip_mask(model, x_train[:64], y_train[:64], sparsity=0.7)

    # Very quick fine-tuning
    quick_train(model, x_train, y_train, x_test, y_test, epochs=30, batch_size=64)

In [ ]:
# 1. Force compatible versions (reset everything to Colab defaults)
!pip install -U --force-reinstall numpy==1.23.5
!pip install -U --force-reinstall tensorflow==2.14.0
!pip install -U tensorflow-model-optimization

# 2. Restart runtime automatically after install
import os
os.kill(os.getpid(), 9)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 98.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
tensorflow 2.18.0 requires numpy<2.1.0,>=1.26.0, but you have numpy 1.23.5 which is incompatible.
blosc2 3.3.0 requires numpy>=1.26, but you have numpy 1.23.5 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
treescope 0.1.9 requires numpy>=1.25.2, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.23.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.9/489.9 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 115.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 106.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 M

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 107.2 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.4
    Uninstalling numpy-2.2.4:
      Successfully uninstalled numpy-2.2.4
  Attempting uninstall: absl-py
    Found existing installation: absl-py 2.2.2
    Uninstalling absl-py-2.2.2:
      Successfully uninstalled absl-py-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jax 0.5.2 requires ml_dtypes>=0.4.0, but you have ml-dtypes 0.2.0 which is incompatible.
tensorflow-text 2.18.1 requires tensorflow<2.19,>=2.18.0, but you have tensorflow 2.14.0 which is incompatible.